# **Deformable Convolution for Lung Cancer Prediction Based on Transcriptomic Data : a Deformable Convolutional Neural Network**
> Author : **Aymen MERROUCHE**. <br>
> In this notebook, we implement a deformable convolutional network for our binary classification task. First we pre train our MLP on the non Lung cancer dataset. Then, we fine tune it on the lung cancer dataset (we don't keep the final classification layer) :

In [32]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import numpy as np
import datetime

from utils import *
from train import *
from data_utils import *
from modules.deformable_cnn import *
from modules.focal_loss import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
# device to use, if cuda available then use cuda else use cpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Working on : ", device)

Working on :  cuda


In [34]:
# load hyperparametrs
# data paths args
with open('./configs/data_paths.yaml', 'r') as stream:
    data_paths_args  = yaml.load(stream,Loader=yaml.Loader)
    
    
# basic cnn args
with open('./configs/def_cnn.yaml', 'r') as stream:
    basic_cnn_args  = yaml.load(stream,Loader=yaml.Loader)

## **1 - Pre-Training on the Non Lung Dataset :**

### **1 - 1 - Get the Data :**

In [26]:
%%time
# Getting the data
# dataset
non_lung_dataset = TranscriptomicImagesDatasetNonLung(data_paths_args["path_to_pan_cancer_hdf5_files"], data_paths_args["path_to_treemap_images"])
non_lung_dataloader_train, non_lung_dataloader_validation = get_data_loaders(non_lung_dataset, batch_size_train = basic_cnn_args["batch_size_pt"],\
                                                                             batch_size_validation = basic_cnn_args["batch_size_pt"])

CPU times: user 4.75 s, sys: 985 ms, total: 5.74 s
Wall time: 2.91 s


### **1 - 2 - Network, Criterion and Training :**

In [ ]:
# network
net = DeformConvNet().to(device).double()

# loss and optimizer  
criterion = FocalLoss(gamma=3).to(device)
optimizer = optim.Adam(net.parameters(), lr=basic_cnn_args['lr_pt'])

# Logging + Experiment

ignore_keys = {'no_tensorboard'}
# get hyperparameters with values in a dict
hparams = {**basic_cnn_args}
# generate a name for the experiment
expe_name = '_'.join([f"{key}={val}" for key, val in hparams.items()])
print("Experimenting with : \n \t"+expe_name)
# path where to save the model
savepath = Path('/tempory/transcriptomic_data/pre_trained_def_cnn_checkpt.pt')
# Tensorboard summary writer
if basic_cnn_args['no_tensorboard']:
    writer = None
else:
    writer = SummaryWriter("runs/runs"+"_"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")+expe_name)
    
# start the experiment
checkpoint = CheckpointState(net, optimizer, savepath=savepath)
fit(checkpoint, criterion, non_lung_dataloader_train, non_lung_dataloader_validation, basic_cnn_args['epochs'], writer=writer)
if not basic_cnn_args['no_tensorboard']:
    writer.close()

Epoch 1/50:   0%|          | 1/646 [00:00<01:12,  8.92it/s, loss=1.0283e-01]

Experimenting with : 
 	epochs=50_batch_size_pt=8_lr_pt=0.001_batch_size_ft=8_lr_ft=0.001_no_tensorboard=True
Training on GPU 



Epoch 2/50:   0%|          | 2/646 [00:00<01:01, 10.47it/s, loss=9.0169e-02]

Epoch 1/50, Train Loss: 8.5232e-02, Test Loss: 0.0901
Epoch 1/50, Train Accuracy: 55.96%, Test Accuracy: 60.36%
Epoch 1/50, Train AUC: 55.97%, Test AUC: 56.59%
Classification Report on Val Set : 
              precision    recall  f1-score   support

   No Cancer       0.90      0.62      0.73      2233
      Cancer       0.16      0.52      0.24       310

    accuracy                           0.60      2543
   macro avg       0.53      0.57      0.49      2543
weighted avg       0.81      0.60      0.67      2543



Epoch 3/50:   0%|          | 2/646 [00:00<01:01, 10.51it/s, loss=8.1974e-02]

...

Epoch 23/50:   0%|          | 2/646 [00:00<01:01, 10.48it/s, loss=4.8833e-02]

Epoch 22/50, Train Loss: 2.3596e-02, Test Loss: 0.0696
Epoch 22/50, Train Accuracy: 82.49%, Test Accuracy: 71.61%
Epoch 22/50, Train AUC: 82.45%, Test AUC: 57.86%
Classification Report on Val Set : 
              precision    recall  f1-score   support

   No Cancer       0.90      0.76      0.82      2233
      Cancer       0.19      0.40      0.25       310

    accuracy                           0.72      2543
   macro avg       0.54      0.58      0.54      2543
weighted avg       0.81      0.72      0.76      2543



Epoch 24/50:   0%|          | 2/646 [00:00<01:01, 10.50it/s, loss=1.7703e-02]

Epoch 23/50, Train Loss: 1.4030e-02, Test Loss: 0.0520
Epoch 23/50, Train Accuracy: 88.41%, Test Accuracy: 75.93%
Epoch 23/50, Train AUC: 88.40%, Test AUC: 57.27%
Classification Report on Val Set : 
              precision    recall  f1-score   support

   No Cancer       0.90      0.82      0.86      2233
      Cancer       0.20      0.33      0.25       310

    accuracy                           0.76      2543
   macro avg       0.55      0.57      0.55      2543
weighted avg       0.81      0.76      0.78      2543



Epoch 24/50:  84%|████████▍ | 542/646 [00:49<00:09, 10.88it/s, loss=2.5638e-03]

## **2 - Fine-Tuning on the Lung dataset :**

### **2 - 1 - Load Pre-Trained Model :**

In [12]:
# Load the pretrained Model
net =  MLP(len(non_lung_dataset[0][0])).to(device).double()
optimizer = optim.Adam(net.parameters(), lr=basic_cnn_args['lr_ft'])
# path where the pre-trained model is saved : defined above+"_best"
savepath = Path('/tempory/transcriptomic_data/pre_trained_def_cnn_checkpt_best.pt')
checkpoint = CheckpointState(net, optimizer, savepath=savepath)
checkpoint.load()
pretrained = checkpoint.model

### **2 - 2 - Get the Data :**

In [ ]:
%%time
# Getting the data
# dataset
lung_dataset = TranscriptomicVectorsDatasetLung(data_paths_args["path_to_pan_cancer_hdf5_files"])
lung_dataloader_train, lung_dataloader_validation = get_data_loaders(lung_dataset, batch_size_train = basic_cnn_args["batch_size_ft"],\
                                                                             batch_size_validation = basic_cnn_args["batch_size_ft"])

### **2 - 3 - Fine Tuning Procedure :**

In [ ]:
# Beginnig Of Transfer Learnig Procedure
net = fine_tune_mlp(pretrained)
net = net.to(device).double()
criterion = FocalLoss().to(device)
optimizer = optim.Adam(net.parameters(), lr=basic_cnn_args['lr_ft'])
savepath = Path('models_finetuned/fine_tuned_mlp_checkpt.pt')
checkpoint = CheckpointState(net, optimizer, savepath=savepath)
fit(checkpoint, criterion, lung_dataloader_train, lung_dataloader_validation, basic_cnn_args['epochs'])